In [1]:
import sys
import os
import pyodbc
import urllib
import pandas as pd
#import com.databricks.spark.avro
from pyspark import  SparkContext
from pyspark import SparkConf
from functools import reduce  # For Python 3.x
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import simplejson as json

server_name = "10.100.60.54"
db_name = "OPTISIMDB_EventSets"
username = "svcCalc"
password = "c@tb0nd"

catrader_folder = "data//catrader//" 
calc_folder = "data//calc//" 

print(sys.executable)
print(sys.path)

sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")

C:\ProgramData\Anaconda3\python.exe
['C:\\Users\\jburns\\AppData\\Local\\Temp\\spark-59224cc8-37ce-4c7e-ae4e-089cbd246550\\userFiles-0d32321b-017b-4dfb-a1fa-5eb16e76f5f0', '', 'C:\\opt\\spark\\spark-2.2.0-bin-hadoop2.7\\python\\lib\\py4j-0.10.4-src.zip', 'C:\\opt\\spark\\spark-2.2.0-bin-hadoop2.7\\python', 'D:\\dev\\notebooks', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\jburns\\.ipython']


In [2]:
def gen_connection_str(
        server_name, db_name, username=None, password=None, raw=False):
    """Generate SQL connection string.

    Args:
        server_name (str): Name of the Azure MSSQL Server
        db_name (str): Name of the Azure MSSQL Database
        username (str): Username for Azure SQL Authentication
        password (str): Password for Azure SQL Authentication
        raw (bool): `True` for raw connection string, `False` for URL format.

    Returns:
        conn_str (str): Connection string

    """
    conn_str = 'DRIVER={};SERVER={};DATABASE={}'. \
        format('ODBC Driver 13 for SQL Server', server_name, db_name)
    if username is not None:
        conn_str += ';UID={0};PWD={1}'.format(username, password)
    if raw:
        return conn_str
    params = \
        urllib.parse.quote_plus(conn_str)
    return 'mssql+pyodbc:///?odbc_connect={0}'.format(params)

    
def read_sql(sql, **kwargs):
    """Read SQL query into a DataFrame.  kwargs are directly pushed to the
        pandas.read_sql function.
    
    Args:        
        sql (str): string SQL query to be executed
        
    """
    conn_str =\
        gen_connection_str(server_name, db_name, username, password, raw=True)
    cnxn = pyodbc.connect(conn_str)
    return pd.read_sql(sql, cnxn, **kwargs)

In [3]:
def read_catrader_eventset_headers(t060_id):
    get_eventset_header_sql = f"""SELECT 
t061_id as EventSetId 
,t061_reference as Name 
,t060_source_system as SourceSystem
,t060_source_version as SourceVersion 
,t061_simulation_count as SimulationCount
,t061_date_imported as ImportDateUtc
,t061_create_user as CreatedByUser
,t061_eventset_xml as AdditionalInfo
,CONCAT(t061_eventset_database,'.',t061_eventset_db_owner,'.',t061_eventset_loss_table) as LossSource
,efc.t052_name as EventFactorClassification
FROM [dbo].[t061_event_set] es
INNER JOIN [OPTISIMDB_Eventsets].[dbo].[t060_event_source] e 
ON es.t060_id = e.t060_id 
INNER JOIN [dbo].[t052_event_factor_classification] efc 
ON efc.t052_id = es.t052_id 
WHERE t061_eventset_source = 'CATRADER_IMPORT' 
AND e.t060_id = {t060_id} 
order by t061_date_imported desc"""
    headers = read_sql(get_eventset_header_sql)
    return headers

def read_calc_eventset_headers(t062_id):
    sql = f"""SELECT 
t061_id as EventSetId
,t061_reference as Name
,t060_source_system as SourceSystem
,t060_source_version as SourceVersion	
,es.t061_eventSet_source as EventSetSource
,esp.t062_id as EventSourcePerspectiveId
,esp.t062_name as EventSourcePerspectiveName
,t061_simulation_count as SimulationCount
,t061_date_imported as ImportDateUtc
,t061_create_user as CreatedByUser
,CONCAT(t061_eventset_database,'.',t061_eventset_db_owner,'.',t061_eventset_loss_table) as LossSource
,CONCAT(t061_eventset_database,'.',t061_eventset_db_owner,'.',t061_eventset_table) as EventSource
FROM [dbo].[t061_event_set] es
INNER JOIN [OPTISIMDB_Eventsets].[dbo].[t060_event_source] e
ON es.t060_id = e.t060_id
INNER JOIN [OPTISIMDB_Eventsets].[dbo].[t062_event_source_perspective] esp
ON esp.t062_id = es.t062_id
WHERE t061_eventset_source = 'CALC'
AND e.t060_id = 44
AND esp.t062_id = {t062_id}
order by t061_date_imported desc"""
    headers = read_sql(sql)
    return headers

def get_catrader_event_set(source):
    sql_query = f"""SELECT 
t031_id as GeoId
 ,el.t010_id as PerilId
 ,t015_id as LobId
 ,el.event_id as EventId
 ,e.event_year as [Simulation]
 ,e.event_day as [Day]	
,el.ground_up_loss_inc_ds as Loss
FROM {source} el
INNER JOIN [OPTISIMDB_Eventsets_AIR18_Vendor].[dbo].[t105670_event] e 
ON el.event_parent_checksum = e.event_checksum
ORDER BY Year, Day, EventId, GeoId, PerilId, LobId"""
    ylt = read_sql(sql_query)
    return ylt

def get_calc_event_set(eventsource, losssource):
    sql_query = f"""SELECT 
t031_id as GeoId	
,el.t010_id as PerilId
,t015_id as LobId
,e.event_id as EventId
,e.event_year as [Simulation]
,e.event_day as [Day]	
,e.event_is_synthetic EventIsSynthetic
,SUM(el.layer_attrition) as LayerAttrition
,SUM(el.layer_attrition_by_year) as LayerAttritionByYear
,SUM(el.reinstatement_attrition) as ReinstatementAttrition
,SUM(el.reinstatement_attrition_by_year) as ReinstatementAttritionByYear
from 
{losssource} el
INNER JOIN {eventsource} e 
ON el.event_checksum = e.event_checksum
GROUP BY
t031_id
,el.t010_id 
,t015_id 
,e.event_id 
,e.event_year
,e.event_day 
,e.event_is_synthetic 
ORDER BY Year, Day, EventId, GeoId, PerilId, LobId"""
    ylt = read_sql(sql_query)
    return ylt

def save_catrader_eventset_as_parquet(lossSource, eventSetId):
    eventset = get_catrader_event_set(lossSource)    
    df = sqlContext.createDataFrame(eventset)
    df.write.parquet(catrader_folder + str(eventSetId))
    print("Written " + str(eventSetId))
    
def save_calc_eventset_as_parquet(eventSource, lossSource, eventSetId):
    eventset = get_calc_event_set(eventSource, lossSource)    
    df = sqlContext.createDataFrame(eventset)
    df.write.parquet(calc_folder + str(eventSetId))
    print("Written " + str(eventSetId))


In [ ]:
ct_headers = read_catrader_eventset_headers(44)

for ct_header in ct_headers.itertuples():
    folder = catrader_folder + str(ct_header.EventSetId) 
    try:        
        if (os.path.isdir(folder)):
            print(f"Skipped {folder}")
            continue
        else:
            print(f"Writing {folder}")
            print(ct_header.LossSource)
            print(ct_header.EventSetId)            
            save_catrader_eventset_as_parquet(ct_header.LossSource, ct_header.EventSetId)                        
            with open(folder + "//header.json", 'w') as f:
                f.write(ct_headers.iloc[[ct_header.Index]].to_json(orient='records'))
    except Exception as err:
        print(f"Error writing {folder} with {err}")
    

Skipped data//catrader//2172194
Skipped data//catrader//2171978
Skipped data//catrader//2171916
Skipped data//catrader//2171913
Skipped data//catrader//2171881
Skipped data//catrader//2169851
Skipped data//catrader//2169789
Skipped data//catrader//2161612
Skipped data//catrader//2161611
Writing data//catrader//2161556
OPTISIMDB_Eventsets_AIR18_Loss3.dbo.t2161556_event_loss
2161556
Error writing data//catrader//2161556 with can not infer schema from empty dataset
Writing data//catrader//2161528
OPTISIMDB_Eventsets_AIR18_Loss2.dbo.t2161528_event_loss
2161528
Error writing data//catrader//2161528 with can not infer schema from empty dataset
Writing data//catrader//2161527
OPTISIMDB_Eventsets_AIR18_Loss3.dbo.t2161527_event_loss
2161527
Error writing data//catrader//2161527 with can not infer schema from empty dataset
Skipped data//catrader//2161463
Skipped data//catrader//2161356
Skipped data//catrader//2161354
Skipped data//catrader//2161346
Skipped data//catrader//2161345
Skipped data//c

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1021, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1025, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Tra

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
  

...

Error writing data//catrader//302312 with An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Writing data//catrader//302311
OPTISIMDB_Eventsets_AIR18_Loss1.dbo.t302311_event_loss
302311


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it


Error writing data//catrader//302311 with An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Writing data//catrader//302310
OPTISIMDB_Eventsets_AIR18_Loss1.dbo.t302310_event_loss
302310


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it


Error writing data//catrader//302310 with An error occurred while trying to connect to the Java server (127.0.0.1:57310)
Writing data//catrader//302273
OPTISIMDB_Eventsets_AIR18_Loss1.dbo.t302273_event_loss
302273


In [ ]:
calc_headers = read_calc_eventset_headers(200)
calc_header = calc_headers.head(1)
print(calc_header.EventSetId.item())
save_calc_eventset_as_parquet(calc_header.EventSource.item(), calc_header.LossSource.item(), calc_header.EventSetId.item())
with open(calc_folder + str(calc_header.EventSetId.item()) + "//header.json", 'w') as f:
    f.write(calc_header.to_json(orient='records'))